In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import string
import re
import nltk
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn import svm
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from collections import Counter
import nltk

In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer
from sklearn.metrics import classification_report


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 92.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 115.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=684d3af55337b447942ff0c416f0f86a26edd717890422aee5829b83933a42af
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


GLOBAL INTIALISATIONS

In [4]:
nltk.download('stopwords')
stopwords_list = stopwords.words('english')
stopwords_list.extend(['nalla','ena','unaku','per','irukanga','panna','yarum','mattum','ivan','ada','pesa','unakku','k','sari','idhu','vida','vittu','enga','yen','ithu','poda','dey','irundhu','ya','la', 'u','r','s','bro','da','dei','dai','nu','ah','nee','ni','illa','un','ok','na','pls','ur','unga']) 
stopwords_list.extend(['🥰','indha','antha','vera','iruka','pola','innum','avan','summa','ellam','thaan','romba','❤️','ana','ama','apdi','ithula','po','evlo','eruku','irukum','nama','enna','va','hi','h','ku','iruku','naa','va','oru','athu','avanga','neenga','tha','en','di','dhan','ne','ella','intha']) 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
label = { 'None-of-the-above': 0, 'Transphobic':1, 'Counter-speech':2, 'Misandry':3,
       'Homophobia':4, 'Hope-Speech':5, 'Xenophobia':6, 'Misogyny':7}

In [6]:
vocabr = []

#Class 1
vocabr.extend(["superstar", "alaiuradhu", "train", "thirunangaigal", "mari", "pichai", "thirunangai", "9no", "soru", "9", "mental", "oy", "paavam"])
vocabr.extend([ "8", "thirunangai", "stop", "train", "ivlo", "kevalamana", "kasu"])
vocabr.extend([ "pavam", "wolaguthela", "night", "ithungaluku", "panam", "seruppala", "edupathal", "arrest", "kaja", "behavior"])
vocabr.extend(["potdu", "amarthirukiravargal", "boy", "aambalaya", "transgender", "thappe", "night", "saputigala", "sila", "picha"])
vocabr.extend(["thittuvargalam", "engaiyum", "ivaga", "tholla", "avoid", "neenga", "viduvalugapuplic", "support", "mudiyala", "komba", "confused", "avanugala", "ariel", "pannanum", "gay", "kaavalthurai", "pothumakkalukku", "pidingigal", "potta", "ukkara"])
#Class 2
vocabr.extend(["pasanga", "poi", "solla", "dowry", "avangaluku", "aana", "pavam", "ponnunga", "avangaluku", "marriage", "yenna", "ha", "yenna", "fulla", "avangaluku", "ethu", "lgbt", "channel", "solla"])
vocabr.extend(["avangaluku", "future", "type", "sankkarraj", "aana", "43", "sonna", "appa", "avangala", "amma", "thappu", "ungalukku", "thappu", "ungalukku", "enaku", "youtube", "thappu", "parents", "fulla", "maadri"])
vocabr.extend(["24jc", "family", "culture", "irukanum", "suthanthiram", "avunga", "ponnu", "kuduthalum", "yaru", "kuda", "sollution", "solla", "kalam", "normal",  "samadhanam", "mudivu","panni"])
#Class 3
vocabr.extend(["thevdiya", "ruben", "potta", "pundaya", "dmk", "ambala", "sunni", "unna", "storm", "avana", "raja", "punda", "porandha", "serupala"])
vocabr.extend(["positive", "paiya", "india", "payale", "breaker", "echa", "ularu", "karanam", "mooditu", "saapdra", "breaker", "mudila", "seeman", "veliya", "nai"])
vocabr.extend(["paiya", "parents", "sethu", "pottu", "adichi", "brama", "madhavadha", "sattam", "sottai", "avana", "pudungi", "thevudiya", "thappa", "perukkuda", "pudikalayadei", "dhevudiya", "oudaey", "mairum", "oruru", "jaathi"])
#Class 4
vocabr.extend(["mudiyathu", "orina", "poda", "homosexual", "muslims", "iruntha", "nature", "first", "level", "kusuvil", "homo", "orula", "lesbian", "baadu", "itha", "gay"])
vocabr.extend(["ipo", "avargal", "super", "naadu", "lgbt", "ippadi", "poola", "maari", "solvanga", "aandava", "friend", "wrong", "ponna", "sax", "podi", "sex", "like"])
vocabr.extend(["paraviyathu", "ipo", "sorry", "illaunga", "adha", "pathi", "loosu", "first", "thambi", "kooda", "girl", "parathesigala", "arasan", "nam", "mavale", "male", "freedomand", "judgement", "poi"])
#Class 5
vocabr.extend(["nala", "manasu", "thirunangaigalin", "sister", "manasu", "vanga", "valgaiyil", "1gram", "akka", "100correct", "thirunangai", "really", "level", "rompa"])
vocabr.extend(["2", "lingaa", "dha", "help", "appreciative", "vanga", "good", "anna", "cute", "anbu", "ungalin", "mari"])
vocabr.extend(["100", "andha", "vaalzthukkal", "mudiyala", "annaavangala", "movie", "uyarvom", "jewels", "ennoda", "sir", "like", "nega", "support", "kathrreena", "solra"])
#Class 6
vocabr.extend(["bomb", "china", "punda", "america", "ithuku", "vatican", "india", "virus", "ila", "world", "peru"])
vocabr.extend(["sudukaadu", "solli", "virusa", "food", "patha", "map", "corona","first", "tamilnadu", "pannuthu", "ai", "yaara", "avana", "serupala", "sam", "potta", "ebdida"])
vocabr.extend(["kandupidikama", "vainga", "nasama", "madakam", "map", "matum", "ipd", "zombie", "daa", "avana", "lab", "plan", "thiravendum", "mavan", "unakkum", "adipensumma", "david", "prachanaye", "pathi"])
#Class 7
vocabr.extend(["girls", "42", "p", "solra", "vara", "ponnungala", "ponnuga", "chai", "pottaingala", "kundu", "koothi", "iva", "nadakakudathu", "gh", "yaarumay", "9", "evalugala", "overloaded", "mooditu"])
vocabr.extend(["kekuravaga", "oaththa", "kooda", "doubt", "lesbiana", "vandha", "ponga", "epo", "ungala", "pottutu", "ivalukaluku", "wb", "ponungala", "vela", "nadatha", "loosu", "vela", "koodhi", "1990"])
vocabr.extend(["peru", "cassia", "pottainale", "thakkale", "mudiyathu", "mooditu", "pandranga", "aaite", "chiillatayaluva", "kollanumdi", "loose", "enaku", "kalyanam", "pondatti", "evalungalvay", "loosu", "kuduimbam", "pooodiii"])

vocabr = list(set(vocabr))

# HELPER FUNCTIONS

In [7]:
def tolower(text):
  return text.lower()

def removepunctuation(text):
    clean="".join([i for i in text if i not in string.punctuation])
    return clean

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def removestopwords(text):
  clean = " ".join([word for word in text.split() if word not in stopwords_list])
  return clean

In [8]:
def preprocess(df):
  df['text']= df['text'].apply(lambda x:tolower(x))
  df['text']= df['text'].apply(lambda x:removepunctuation(x))
  df['text']= df['text'].apply(lambda x:deEmojify(x))
  df['text']= df['text'].apply(lambda x:removestopwords(x))
  df['category'] = df['category'].map(label)
  return df

In [9]:
def tokenize(text):
  return text.split()

def tokenizedf(df):
  df['token']= df['text'].apply(lambda x:tokenize(x))
  df['len'] = df.text.apply(lambda x : len(x))
  return df

In [10]:
def stats(value):
    if value == 0:
        return 0
    else:
        return 1

In [11]:
def vocab(df):
  return df.groupby('category')['text'].apply(lambda x: Counter(" ".join(x).split()).most_common(100))

In [12]:
def getVocab(df):
  df_vocab = vocab(df)
  processed = list(set([key for groups in df_vocab for key, value in groups]))
  return processed

In [13]:
def fitTfidfVectrain(df, tfidf_vec):
  x_df = tfidf_vec.fit_transform(df['text'])
  return x_df

In [14]:
def getXtrain(model, df, tfidf_vec):
  model_tf_x = fitTfidfVectrain(df, tfidf_vec)
  model_val_x = model.encode(df['text'].tolist())
  Xval = [np.append(list1, list2) for list1, list2 in zip(model_val_x, model_tf_x.toarray())]
  return Xval

In [15]:
def fitTfidfVectest(df, tfidf_vec):
  x_df = tfidf_vec.transform(df['text'])
  return x_df

In [16]:
def getXtest(model, df, tfidf_vec):
  model_tf_x = fitTfidfVectest(df, tfidf_vec)
  model_val_x = model.encode(df['text'].tolist())
  Xval = [np.append(list1, list2) for list1, list2 in zip(model_val_x, model_tf_x.toarray())]
  return Xval

In [17]:
def getY(df):
  return df['category']

# MODELS

In [18]:
labse_embedding_model = SentenceTransformer('sentence-transformers/LaBSE')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/114 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.62M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.22M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/461 [00:00<?, ?B/s]

# DATASETS AND PREPROCESSING

In [19]:
df_train = pd.read_csv('/content/drive/Shareddrives/NLP Research work - PANDAS team/ACL/Datasets/ACD/ta-en-misogyny-train.csv',sep='\t', names = ["category", "text"])
df_train.text=df_train.text.astype(str)
train = df_train
train = preprocess(df_train)

In [20]:
df_test = pd.read_csv('/content/drive/Shareddrives/NLP Research work - PANDAS team/ACL/Datasets/ACD/ta-en-misogyny-dev.csv',sep='\t', names = ["category", "text"])
df_test.text=df_test.text.astype(str)
test = df_test
test = preprocess(test)

In [21]:
train

,category,text
0,0,enaku unmaikum aluha wantu thirunangaigal thei...
1,1,superstar vijay arivuketta polu thappu pavam a...
2,0,ugka smile cute
3,0,anna waiting 🥰🥰🥰
4,0,yanda tamilnadu evvalavo pirachana athalam vit...
...,...,...
5943,0,noq day caste religious certificate
5944,0,mimicry escape aitaru
5945,0,rajesh age
5946,2,videos nallarukku dont prank eena varavanga ep...


In [22]:
test

,category,text
0,0,black saree hot
1,2,halo first ayunga kai thatti amount keatkyrang...
2,0,hello manithana sollunga pramanana solla vanam
3,6,china kaaranai kalaipan daily enjoying 2016
4,0,onnota ponnu ipdi vituviya
...,...,...
1483,1,9 usu vinoth
1484,0,love panuradhaaaa soliyaeeee cover panura niiii
1485,0,part 2 aippa varum
1486,3,jii nariya thetunga andha potta gopiya


#VOCABULARY AND FEATURE EXTRACTION

In [23]:
vocab = getVocab(train)
#to include new vocabulary
vocab.extend(vocabr)
vocab = list(set(vocab))

#TRAINING

In [24]:
tfidf_vec = TfidfVectorizer(analyzer = 'word', vocabulary = vocab)

In [25]:
Xtrain = getXtrain(labse_embedding_model, train, tfidf_vec)
Xtest = getXtest(labse_embedding_model, test, tfidf_vec)

In [26]:
y_train = getY(train)
y_test = getY(test)

Original

In [27]:
svm_model = svm.SVC(kernel='linear', C=1, verbose=True, probability = True)
svm_model.fit(Xtrain, y_train)

[LibSVM]

SVC(C=1, kernel='linear', probability=True, verbose=True)

In [28]:
ypred = svm_model.predict(Xtest)
print(classification_report(y_test, ypred))

              precision    recall  f1-score   support

           0       0.76      0.96      0.85       919
           1       0.60      0.30      0.40        40
           2       0.51      0.34      0.41        95
           3       0.72      0.54      0.62       218
           4       0.68      0.30      0.42        43
           5       0.40      0.08      0.13        53
           6       0.82      0.57      0.67        70
           7       0.64      0.18      0.28        50

    accuracy                           0.74      1488
   macro avg       0.64      0.41      0.47      1488
weighted avg       0.72      0.74      0.71      1488



Linear model with Intercept

In [29]:
svm_model = svm.LinearSVC(C=1, verbose=True, fit_intercept = True, intercept_scaling = 1)
svm_model.fit(Xtrain, y_train)
ypred = svm_model.predict(Xtest)
print(classification_report(y_test, ypred))

[LibLinear]              precision    recall  f1-score   support

           0       0.79      0.92      0.85       919
           1       0.38      0.28      0.32        40
           2       0.46      0.28      0.35        95
           3       0.67      0.60      0.63       218
           4       0.54      0.35      0.42        43
           5       0.35      0.15      0.21        53
           6       0.77      0.67      0.72        70
           7       0.58      0.28      0.38        50

    accuracy                           0.74      1488
   macro avg       0.57      0.44      0.48      1488
weighted avg       0.71      0.74      0.72      1488



Linear model with modified Class Weights

In [30]:
svm_model = svm.LinearSVC(C = 1, class_weight = {0: 0.09, 1: 0.13, 2: 0.13, 3: 0.13, 4: 0.13, 5: 0.13, 6: 0.13, 7: 0.13}, verbose = True, fit_intercept = True, intercept_scaling = 1)
svm_model.fit(Xtrain, y_train)
ypred = svm_model.predict(Xtest)
print(classification_report(y_test, ypred))

[LibLinear]              precision    recall  f1-score   support

           0       0.81      0.89      0.85       919
           1       0.34      0.28      0.31        40
           2       0.42      0.35      0.38        95
           3       0.61      0.61      0.61       218
           4       0.57      0.40      0.47        43
           5       0.35      0.23      0.28        53
           6       0.68      0.69      0.68        70
           7       0.50      0.24      0.32        50

    accuracy                           0.73      1488
   macro avg       0.54      0.46      0.49      1488
weighted avg       0.71      0.73      0.71      1488

